# Exploratory analysis of the dataset

Banter and more banter

https://www.kaggle.com/notgibs/500-greatest-albums-of-all-time-rolling-stone?select=albumlist.csv

country_codes= ['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from enrich_functions import *
from analysis_functions import *
import re

## Data cleaning

In [2]:
top500_raw = pd.read_csv("../dataset/albumlist.csv", index_col=0, encoding = "ISO-8859-1")
top500_raw.head()

,Year,Album,Artist,Genre,Subgenre
Number,,,,,
1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [3]:
top500_raw.columns = map(str.lower, top500_raw.columns)

Separate generes and subgeneres in different columns.

In [4]:
clean = lambda x: x.replace("Ê"," ").split(", ")
top500_raw["genre"]=top500_raw["genre"].apply(clean)
top500_raw["subgenre"]=top500_raw["subgenre"].apply(clean)

Renaming wrong album names and dropping the ones that arent in spoty

In [5]:
def wrong(x):
    dic={'The Beatles ("The White Album")':"The White Album",
          'Sign "Peace" the Times':'Sign "O" the Times',
          'The Band ("The Brown Album")':'The Band',
          'Back to Mono (1958-1969)':'Back To Mono',
          '[Led Zeppelin IV]':'Led Zeppelin IV',
          'Saturday Night Fever: The Original Movie Sound Track':'Saturday Night Fever',
          "The B 52's / Play Loud":"The B 52's",
          "The Very Best of Linda Ronstadt":"The Best of Linda Ronstadt",
          "The Anthology: 1961-1977": "The Very Best of Curtis Mayfield",
          'Blues Breakers With Eric Clapton ("The Beano Album")':"Bluesbreakers",
          "Bo Diddley / Go Bo Diddley":"Go Bo Diddley",
          "Can't Buy a Thrill":"Cant Buy a Thrill",
          'Metallica ("The Black Album")':"Metallica",
          "Trans Europa Express":"Trans-Europe Express",
          "'":"",
          "Honky Chteau":"Honky Chateau",
          "Cheap Trick at Budokan":"At Budokan"
          }
    xr=x
    for wrong,right in dic.items():
        xr=xr.replace(wrong,right)
    return xr
#wrong = lambda x: x.replace(wrong, right) for wrong, right in dict_rep.items()
top500_raw["album"]=top500_raw["album"].apply(wrong)


In [6]:
top500_raw.drop([178,196,198,214,245,278,289,346,363,377,400,421,449,170,459],inplace=True)

Enrich spoty

In [7]:
load_dotenv()
cid = os.getenv("SPOTIFY_ID")
secret = os.getenv("SPOTIFY_SECRET")
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
spotify

In [8]:
features_median = lambda x: get_album_features_median(spotify,x)


In [12]:
top500_raw["features"]=top500_raw["album"].apply(features_median)

KeyboardInterrupt: 

In [ ]:
top500_raw=pd.concat([top500_raw, top500_raw["features"].apply(pd.Series)], axis=1, sort=False)

get_wiki_views(lang,page,ini_year,ini_month,ini_day,end_year,end_month,end_day)

In [ ]:
wiki= lambda x: get_wiki_views("es",x,2019,10,1,2020,10,1)
top500_raw["wiki_views_album"]=top500_raw["album"].apply(wiki)
top500_raw["wiki_views_artist"]=top500_raw["artist"].apply(wiki)

In [ ]:
top500_raw.to_csv('../dataset/albumlist_clean.csv')